In [2]:
import base64
import requests

CLIENT_ID = 'AbvyUIwp3glBnPfxyEEgr9K_7QTT-TrPnV1MsCn_JkyxDFsUoJ_ITjFrX592jpIfzlWEGsM2GajfSV4O'
SECRET = 'EBhnnkkzN9zpSRG0ACTBmgk0GoeIwmPGelNA1MrdFe7zOlE4ApZXAmnq0jovkjwHBZPg-qQilUokXePL'

PAYPAL_API = 'https://api-m.sandbox.paypal.com'


def get_access_token():
    auth = base64.b64encode(f"{CLIENT_ID}:{SECRET}".encode()).decode()
    headers = {
        'Authorization': f'Basic {auth}',
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    data = {'grant_type': 'client_credentials'}

    response = requests.post(f"{PAYPAL_API}/v1/oauth2/token", headers=headers, data=data)
    response.raise_for_status()
    return response.json()['access_token']


def create_product(access_token):
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
    }
    product_data = {
        "name": "VIP Subscription",
        "description": "Subscription to VIP status",
        "type": "SERVICE",
        "category": "SOFTWARE"
    }
    response = requests.post(f"{PAYPAL_API}/v1/catalogs/products", headers=headers, json=product_data)
    response.raise_for_status()
    return response.json()


def create_plan(access_token, product_id):
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
    }
    plan_data = {
        "product_id": product_id,
        "name": "Monthly VIP Plan",
        "billing_cycles": [
            {
                "frequency": {
                    "interval_unit": "MONTH",
                    "interval_count": 1
                },
                "tenure_type": "REGULAR",
                "sequence": 1,
                "total_cycles": 0,
                "pricing_scheme": {
                    "fixed_price": {
                        "value": "10",
                        "currency_code": "USD"
                    }
                }
            }
        ],
        "payment_preferences": {
            "auto_bill_outstanding": True,
            "setup_fee_failure_action": "CONTINUE",
            "payment_failure_threshold": 3
        },
        "taxes": {
            "percentage": "0",
            "inclusive": False
        }
    }
    response = requests.post(f"{PAYPAL_API}/v1/billing/plans", headers=headers, json=plan_data)
    response.raise_for_status()
    return response.json()


def main():
    print("Getting access token...")
    token = get_access_token()
    print("Access Token:", token)

    print("Creating product...")
    product = create_product(token)
    print("Product created:", product)

    print("Creating plan...")
    plan = create_plan(token, product['id'])
    print("Plan created:", plan)


if __name__ == "__main__":
    main()


Getting access token...
Access Token: A21AALErW_Ci8poWGpi8EgzzwUtxjf6BKBRWov-_gyP7bnr8JEaQC6UhZcUhiMcma0CeI9T7VhiV-tLDg2D5u8_TZMRyRS1_A
Creating product...
Product created: {'id': 'PROD-8TV75457AC816752K', 'name': 'VIP Subscription', 'description': 'Subscription to VIP status', 'create_time': '2025-05-21T14:47:13Z', 'links': [{'href': 'https://api.sandbox.paypal.com/v1/catalogs/products/PROD-8TV75457AC816752K', 'rel': 'self', 'method': 'GET'}, {'href': 'https://api.sandbox.paypal.com/v1/catalogs/products/PROD-8TV75457AC816752K', 'rel': 'edit', 'method': 'PATCH'}]}
Creating plan...
Plan created: {'id': 'P-5JK91908E08174815NAW6O4Q', 'product_id': 'PROD-8TV75457AC816752K', 'name': 'Monthly VIP Plan', 'status': 'ACTIVE', 'usage_type': 'LICENSED', 'create_time': '2025-05-21T14:47:14Z', 'links': [{'href': 'https://api.sandbox.paypal.com/v1/billing/plans/P-5JK91908E08174815NAW6O4Q', 'rel': 'self', 'method': 'GET', 'encType': 'application/json'}, {'href': 'https://api.sandbox.paypal.com/v1/bil